In [1]:
from sentence_transformers import SentenceTransformer
import json
import os
from sklearn.metrics.pairwise import cosine_similarity


In [2]:
# see which model to use
model_name = 'sentence-transformers/all-mpnet-base-v2'
model = SentenceTransformer(model_name)

In [6]:
DATASET_LOCATION = "../data/"
with open(os.path.join(DATASET_LOCATION, "commits.json"), "r") as f:
    commit_data = json.load(f)
with open(os.path.join(DATASET_LOCATION, "issues.json"), "r") as f:
    issue_data = json.load(f)

In [8]:
commit_sentences = []
issue_sentences = []

for commit_obj in commit_data:
    commit_sentences.append(commit_obj["commit"])

for issue_obj in issue_data:
    if(issue_obj["body"] !=None):
        issue_sentences.append(issue_obj["title"]+" "+issue_obj["body"])
    else:
        issue_sentences.append(issue_obj["title"])


In [12]:
print(len(commit_sentences),len(issue_sentences))

2196 274


In [13]:

issue_embeddings =  model.encode(issue_sentences)


In [14]:
commit_embeddings =  model.encode(commit_sentences)

In [26]:
result_obj = {}
corr = cosine_similarity(issue_embeddings,commit_embeddings)
for i in range(len(issue_embeddings)):
    issue = issue_sentences[i]
    result_obj[issue] = {}
    for j in range(len(commit_embeddings)):
        commit =  commit_sentences[j]
        result_obj[issue][commit] = str(corr[i][j])


In [28]:
corr

array([[ 1.7806265e-01,  1.5512379e-01,  1.8297099e-01, ...,
         1.2196938e-01,  1.3470286e-01,  2.1449994e-02],
       [ 2.3876302e-01,  2.8768498e-01,  2.4566457e-01, ...,
         1.6324857e-01, -6.5639354e-03,  6.8825044e-02],
       [ 3.6080176e-01,  1.4636503e-01,  2.3723969e-01, ...,
         2.6378021e-01,  3.5880152e-02,  4.5071065e-02],
       ...,
       [ 3.0068722e-01,  1.1551815e-01,  2.4902582e-01, ...,
         3.3448905e-01, -1.9292101e-02, -7.5622927e-03],
       [ 2.8679109e-01,  1.5653782e-01,  1.4433992e-01, ...,
         2.9501578e-01,  6.6696182e-02,  1.4214226e-02],
       [ 2.4157140e-01,  1.8778610e-01,  3.7034312e-01, ...,
         3.4054130e-01,  8.5204635e-03,  3.3092313e-04]], dtype=float32)

In [27]:
# with open('corr_issue_commit.json', 'w') as f:
#     json.dump(result_obj, f, indent=4)

In [36]:
import numpy as np

In [70]:
def get_n_related_commits(issue,n):
    
    issue_embedding = model.encode([issue])
    cosine = cosine_similarity(issue_embedding,commit_embeddings)
    
    ind = np.argpartition(cosine[0], -n)[-n:]
    ans=[]
    for i in ind:
        # print(i)
        ans.append(commit_sentences[int(i)])
    return ans

In [71]:
get_n_related_commits("repository url",10)

["Populates Issue's repository from URL when not in response.",
 'fix repository',
 'RepositoryKey: first step',
 'Repository docstring',
 'Repository',
 'Repository.xxx_download',
 'Repository.edit',
 'RepositoryDetails',
 'Repository issues',
 'RepositoryKeys']